In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import gymnasium as gym
import matplotlib.pyplot as plt
from PIL import Image
from reward import Reward
from helper import detect, capture, pad_inner_array, SocketListener, DataQueue
from pynput import keyboard
from pynput.keyboard import Controller, Key
from concurrent.futures import ThreadPoolExecutor
from collections import deque
from a2c_agent import A2C_Agent
import mss
import time
import pathlib
import os
import json
import random
import logging
import warnings
import sys

In [2]:
if not os.path.exists('yolov5'):
    !git clone https://github.com/ultralytics/yolov5
    !pip install -r yolov5/requirements.txt

warnings.simplefilter("ignore", FutureWarning)
logging.getLogger('ultralytics').setLevel(logging.ERROR)

In [3]:
class AC_Net(nn.Module):
  def __init__(self, input, action_space, lstm_layers=1):
    super(AC_Net, self).__init__()
    self.dropout = nn.Dropout(0.2)
    self.lstm = nn.LSTM(input[1] * input[2], 128, num_layers=lstm_layers, batch_first=True, dropout=0.2, bidirectional=True)
    
    self.actors = nn.ModuleList([
        nn.Sequential(
          nn.Linear(256, 256),
          nn.ReLU(),
          nn.Linear(256, action.n),
        )
        for action in action_space
      ])
    self.critic = nn.Linear(256, 1)
    
    # for layer in [self.fc1, self.fc2, *self.actors, self.critic]:
    #   nn.init.xavier_uniform_(layer.weight)
    #   nn.init.constant_(layer.bias, 0)
    
  def forward(self, x, hx=None):
    x = x.view(x.size(0), -1, x.size(2) * x.size(3))
    x, hx = self.lstm(x, hx)
    x = x[:, -1, :]
    x = self.dropout(x)
     
    actor_logits = torch.stack([actor(x) for actor in self.actors], dim=1)
    return actor_logits, self.critic(x), hx

In [4]:
class SimOsuEnvironment(gym.Env):
  def __init__(self, stacked_frames=1, max_notes=8, random_select_from_metadata=True, render_mode=True, normalized_rewards=True):
    '''
    args:
      stacked_frames: n frames to stack for observation
      max_notes: n notes with the highest y_center pos to be considered per frame in the observation
      random_select_from_metadata: Randomly select a song from the metadata file for render mode
      render_mode:  The environment will interactive with the game, requires window to be in focus and in song selection screen. 
        Otherwise, simulated mode with pre-recorded frames will be used
      normalized_rewards: Normalizes the rewards based on the number of notes in the song
    '''
    # Common parameters
    self.max_notes = max_notes
    self.stacked_frames = stacked_frames
    self.random_select_from_metadata = random_select_from_metadata
    self.normalized_rewards = normalized_rewards
    self.render_mode = render_mode
    self.keys_reference = ['s', 'd', 'k', 'l'] # Used for keyboard input with index corresponding to a lane
    self.observation_space = gym.spaces.Box(low=0, high=np.inf, shape=(self.stacked_frames, self.max_notes, 3))
    self.action_space = gym.spaces.MultiDiscrete([4] * len(self.keys_reference))
    self.observation = np.zeros((self.max_notes, 3))
    self.frame_buf = deque(maxlen=self.stacked_frames)
    self.terminated = False
  
    self.invalid_actions = 0 # Counts the number of invalid actions in a step
    self.invalid_action_penalty = -0.1
    self.invalid_action_penalty_scaled = 0 # Scaled invalid action penalty, to be set in reset
    self.total_invalid_actions = 0
    self.actions_taken = np.zeros((4, 4))
    
    self.executor = ThreadPoolExecutor()
    self.reward_func = Reward()
    self.model = None 
    
    self.frame_notes = [] # A note representation of each frame
    
    # Parameters for render mode
    if render_mode:
      self.capture_region = None
      self.keyboard = Controller()
      self.data_queue = DataQueue()
      self.listener = None
      self.listener = SocketListener()
      self.listener.start(data_handler=self.data_queue.add, traceback=False)
      
    self._vision_setup()
    
    # Parameters for simulated mode
    self.custom_reward = [8, 10, 1, -3, -2, -1, -1, -2, -1] # Custom reward function for simulated mode
    self.chosen_folder = None # A randomly chosen folder from available_folders, to be set in reset
    self.available_folders = os.listdir('./frames/rl_simulated_training_sample')
    self.frames = [] # A list of frames in the chosen folder, to be set in reset
    self.metadata = { # Metadata of the chosen song, to be set in reset
      'song_name': "No metadata",
      'song_duration': 0,
      'note_count': 0,
      'hold_note_count': 0,
      'difficulty': 0
    }
    self.frame_step = 0
    self.note_count = 0
    
  def reset(self):
    '''
    Returns to the initial state of the environment and selects a new song
    
    (Warning: Keyboard actions are performed in render_mode on reset, the game should be in focus and in song selection screen)
    '''
    for _ in range(self.stacked_frames):
      self.frame_buf.append(np.zeros((self.max_notes, 3), dtype=np.int32))
    self.observation = np.stack(self.frame_buf, axis=0)
    self.invalid_actions = 0
    self.total_invalid_actions = 0
    self.actions_taken = np.zeros((4, 4))
    self.terminated = False
    
    if self.render_mode:
      if self.listener.has_connection:
        print('Closing connection')
        self.listener.close_connection()
      
      if self.random_select_from_metadata:
        self.metadata = json.load(open('./song_metadata.json'))
        self.metadata = random.choice(self.metadata)
        
      self.data_queue.clear()
      self._get_random_song()
    else:
      random_folder = random.choice(self.available_folders)
      self.chosen_folder = './frames/rl_simulated_training_sample/' + random_folder
      self.frames = sorted(os.listdir(self.chosen_folder))[:-1]
      self.frame_notes = []
      self.frame_step = 0
      
      self.metadata = json.load(open(self.chosen_folder + '/metadata.json'))
      self.reward_func.set_custom_rewards(self.custom_reward)
    
    return self.observation
  
  def step(self, multi_actions):
    if self.render_mode:
      self.observation, reward = self._render_mode_step(multi_actions)
    else:
      self.observation, reward = self._simulated_mode_step(multi_actions)
    
    self.frame_buf.append(self.observation)
    self.observation = np.stack(self.frame_buf, axis=0)
    
    if self.normalized_rewards:
      self.note_count = self.metadata['note_count'] + self.metadata['hold_note_count'] * 2
      reward = ((reward + self.invalid_actions) / self.note_count) * 100
    
    self.total_invalid_actions += self.invalid_actions
    self.invalid_actions = 0
    info = {}
    
    return self.observation, reward, self.terminated, info
    
  def get_meta_data(self):
    '''
    Returns the metadata of the current song in the form of 
    {
      'song_name' (str), \\
      'song_duration' (int) in seconds, \\
      'note_count' (int,) \\
      'hold_note_count' (int), \\
      'difficulty' (float)
    }
    '''
    return self.metadata
  
  def _render_mode_step(self, multi_actions):
    '''
    Steps the environment in render mode, where the game is running and the game window is visible
    '''
    while self.listener.is_first_connection and not self.listener.has_connection:
      pass
    
    img = capture(self.capture_region)
    img = np.array(img)
    
    self._update_observation(img)
    
    actions, action_types = self._parse_multi_actions(multi_actions)
    self.executor.submit(self._perform_keyboard_action, actions, action_types)
    
    data = self.data_queue.get()
    self.data_queue.clear()
    reward = self.reward_func.get_in_game_reward(data) + self.invalid_action_penalty * self.invalid_actions
    
    if not self.terminated:
      self.terminated = (not self.listener.is_first_connection and not self.listener.has_connection) or 6 in data or 7 in data
    
    return self.observation, reward
  
  def _simulated_mode_step(self, multi_actions):
    '''
    Steps the environment in simulated mode, where the pre-recorded frames are used as input
    '''
    img = Image.open(self.chosen_folder + '/' + self.frames[self.frame_step])
    self.frame_step += 1
    
    self._update_observation(img)
    
    actions, _ = self._parse_multi_actions(multi_actions)
    reward = self.reward_func.get_simulated_reward(actions, self.frame_notes[-2] if len(self.frame_notes) > 3 else []) + self.invalid_action_penalty_scaled * self.invalid_actions
    self.terminated = self.frame_step >= len(self.frames)
    
    return self.observation, reward
  
  def _update_observation(self, img):
    vision_thread = self.executor.submit(detect, img, self.model, normalize=True)
    self.observation = vision_thread.result()
    self.frame_notes.append(self.observation)
    self.observation = pad_inner_array([self.observation], [0, 0 ,0], self.max_notes)[0]
    
  def _parse_multi_actions(self, multi_actions):
    '''
    Parses the multi actions into a list of actions and their types
    '''
    parsed_actions = []
    action_type = []
    
    for action_lane, action in enumerate(multi_actions):
      self.actions_taken[action_lane][action] += 1
      
      if action == 0: # Do nothing
        pass
      elif action == 1: # Release
        if not self.reward_func.get_key_held(action_lane):
          self.invalid_actions += 1
          continue
        
        self.reward_func.update_keys_held(action_lane, False)
        parsed_actions.append(action_lane)
      elif action == 2: # Press
        if self.reward_func.get_key_held(action_lane):
          self.invalid_actions += 1
          
        self.reward_func.update_keys_held(action_lane, False)
        parsed_actions.append(action_lane)
      elif action == 3: # Hold
        if not self.reward_func.get_key_held(action_lane):
          parsed_actions.append(action_lane)
        
        self.reward_func.update_keys_held(action_lane, True)

      if action_type != 0:
        action_type.append(action)
          
    return parsed_actions, action_type
  
  def _perform_keyboard_action(self, parsed_actions, action_type):
    for action, action_type in zip(parsed_actions, action_type):
      if action_type in [2, 3]:
        self.keyboard.press(self.keys_reference[action])
        if action_type == 3:
          self.reward_func.update_keys_held(action, True)
      
      if action_type in [1, 2]:
        self.keyboard.release(self.keys_reference[action])
        if action_type == 1:
          self.reward_func.update_keys_held(action, False)
          
  def _get_random_song(self):
    '''
    Choose a random song using in-game menu navigation, game must be in focus and in song selection screen
    '''
    if self.random_select_from_metadata:
      key_sequence = ['a', Key.esc, 'a', Key.esc, *list(self.metadata['song_name']), Key.enter]
    else:
      key_sequence = ['a', Key.esc, Key.f2] + ([Key.down] * np.random.randint(0, 6)) + [Key.enter]
    
    time.sleep(2) 
    for key in key_sequence:
      self.keyboard.press(key)
      time.sleep(0.05)
      self.keyboard.release(key)
    
    time.sleep(3)
    self.executor.submit(self._skip_intro)
  
  def _skip_intro(self):
    for _ in range(10):
      self.keyboard.press(Key.space)
      time.sleep(0.05)
      self.keyboard.release(Key.space)
      time.sleep(0.1)
   
  def _vision_setup(self):
    '''
    Sets up the vision model for object detection, also sets up capture region for render mode
    '''
    if os.name == 'nt':
      pathlib.PosixPath = pathlib.WindowsPath # https://github.com/ultralytics/yolov5/issues/10240#issuecomment-1662573188
    
    if self.render_mode:
      monitor = mss.mss().monitors[-1]
      t, l, w, h = monitor['top'], monitor['left'], monitor['width'], monitor['height']
      self.capture_region = {'left': l+int(w * 0.338), 'top': t, 'width': w-int(w * 0.673), 'height': h} 
      
    self.model = torch.hub.load('ultralytics/yolov5', 'custom', path='./models/best.pt', force_reload=True)
    

In [5]:
osu_env = SimOsuEnvironment(max_notes=4, stacked_frames=8, render_mode=True, normalized_rewards=True)

Listening on 127.0.0.1:5555


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\tiany/.cache\torch\hub\master.zip
YOLOv5  2024-12-4 Python-3.10.6 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 4096MiB)

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [6]:
device = torch.device("cuda")
ac_net = AC_Net(osu_env.observation_space.shape, osu_env.action_space, lstm_layers=2).to(device)
#ac_net.load_state_dict(torch.load('model_results/ac_net.pth'))

optimizer = torch.optim.Adam(ac_net.parameters(), lr=0.001)
agent = A2C_Agent(ac_net, osu_env, optimizer, batch_size=128, device=device)

agent.pretrain(max_episode=50)

Found [211527, 9546, 28411, 26372] actions
Found hit types Miss: 415, Meh: 165, Ok: 445, Good: 2269, Great: 11866, Perfect: 16947, Passed: 14, Failed: 0
Using weights for CEL: tensor([ 1.30412, 28.89755,  9.70948, 10.46019], device='cuda:0')
Episode 0     Loss:     1.0329, Accuracy:     0.1521
Episode 1     Loss:     0.7211, Accuracy:     0.2219
Episode 2     Loss:     0.5632, Accuracy:     0.2845
Episode 3     Loss:     0.4756, Accuracy:     0.3351
Episode 4     Loss:     0.4198, Accuracy:     0.3759
Episode 5     Loss:     0.3831, Accuracy:     0.4609
Episode 6     Loss:     0.3457, Accuracy:     0.5241
Episode 7     Loss:     0.3202, Accuracy:     0.5686
Episode 8     Loss:     0.3026, Accuracy:     0.6014
Episode 9     Loss:     0.2856, Accuracy:     0.6262
Episode 10    Loss:     0.2698, Accuracy:     0.6466
Episode 11    Loss:     0.2575, Accuracy:     0.6633
Episode 12    Loss:     0.2437, Accuracy:     0.6773
Episode 13    Loss:     0.2332, Accuracy:     0.6894
Episode 14    Lo

KeyboardInterrupt: 

In [ ]:
#torch.save(ac_net.state_dict(), 'model_results/ac_net_2.pth')

In [ ]:
train_total_rewards = agent.train(max_episode=250)

Action time: 1733329176.8239362
Action time: 0.23384571075439453
Action time: 0.04344916343688965
Action time: 0.04008889198303223
Action time: 0.03992891311645508
Action time: 0.04210662841796875
Action time: 0.04330897331237793
Action time: 0.03928709030151367
Action time: 0.039843082427978516
Action time: 0.042088985443115234
Action time: 0.04293489456176758
Action time: 0.037682294845581055
Action time: 0.04828071594238281
Action time: 0.035928964614868164
Action time: 0.03707075119018555
Action time: 0.039138078689575195
Action time: 0.041951656341552734
Action time: 0.03625774383544922
Action time: 0.045538902282714844
Action time: 0.043453216552734375
Action time: 0.03841876983642578
Action time: 0.041738033294677734
Action time: 0.042786598205566406
Action time: 0.03928875923156738
Action time: 0.04380083084106445
Action time: 0.03840017318725586
Action time: 0.041509389877319336
Action time: 0.039810895919799805
Action time: 0.04501032829284668
Action time: 0.05270886421203613

KeyboardInterrupt: 

In [ ]:
test_total_rewards = agent.test(max_episode=1)

Connection closed.
Episode 0     [Ahoy!! Warera Houshou Kaizokudan] 2.93            Reward:   -17.3022
{'miss': 854, 'bad': 204, 'meh': 507, 'ok': 2353, 'great': 11904, 'perfect': 16956, 'song_cleared': 14, 'song_failed': 0}
152
[[        808          78         114         290]
 [        712          76         109         393]
 [        728          89         108         365]
 [        881          81          77         251]]


In [ ]:
#torch.save(ac_net.state_dict(), 'model_results/ac_net.pth')

In [ ]:
%matplotlib inline
plt.plot(train_total_rewards)
plt.xlabel('Episode')
plt.ylabel('Total Reward')
plt.show()

NameError: name 'train_total_rewards' is not defined